In [1]:
using BenchmarkTools
using Random

const SAFETY = true

function not!(a::AbstractArray, t=1)
    a[t] = !a[t]
    return a
end

cnot(x, y) = x, x ⊻ y

function cnot!(a::AbstractArray, c=1, t=2)
    if (SAFETY && t == c)
         throw(DomainError((c, t),
                "Forbidden Operation.\nControls and targets must have distinct indices!"))
    end
        a[t] = a[t] ⊻ a[c]
    return a
end

toff(x, y, z) = x, y, z ⊻ (x && y)

function toff!(a::AbstractArray, c1=1, c2=2, t=3)
    if (SAFETY && (t == c1 || t ==c2))
        throw(DomainError((c1, c2, t),
              "Forbidden Operation.\nControls and targets must have distinct indices!"))
    end
        a[t] = a[t] ⊻ (a[c1] && a[c2])
    return a
end

fred(x, y, z)=x, y ⊻ (x||(y ⊻ z)), z ⊻ (x&&(y ⊻ z))

function fred!(a::AbstractArray, c=1, t1=2, t2=3)
     if (SAFETY && (c == t1 || c == t2))
        throw(DomainError((c, t1, t2),
            "Forbidden Operation.\nControls and targets must have distinct indices!"))
    end
    if a[c]
        a[[t1; t2]] = a[[t2; t1]]
    end
    return a
end

fred! (generic function with 4 methods)

In [34]:
const circuitDict = Dict('t'=>toff!, 'c'=>cnot!, 
    'n'=>not!, 'f'=>fred!)

function parsegate(str)
    gateregex = r"\d+"
    key = first(str)
    numargs = eachmatch(gateregex, str)
    #ToDo: maybe compare performance with map! function
    numargs = map(x -> parse(Int, x.match), numargs)
    return (key, numargs...)
end

function parsecircuit(str)
    circuitregex = r"[a-zA-Z][^a-zA-Z]*"
    circuitlist = Tuple{Char,Vararg{Int}}[]
    for i in eachmatch(circuitregex, str)
        gate = parsegate(i.match)
        push!(circuitlist, gate)
    end
    return circuitlist
end

dogate!(A, gate) = circuitDict[gate[1]](
                                    A, gate[2:end]...)

dogate!(A, gate::String) = dogate!(A, parsegate(gate)) 

function docircuit!(A, circuit)
    for i in circuit
        dogate!(A, i)
    end
    return A
end
    
docircuit!(A, circ::String) =
docircuit!(A, parsecircuit(circ))

docircuit! (generic function with 2 methods)

In [118]:
function chuang(x,y,z)
beg = Bool[x; y; z]
beg |> toff! |> cnot! |> not!
return beg
end
chuang!(a::AbstractArray) = a |> toff! |> cnot! |> not!
invchuang(a::AbstractArray) = a |> not! |> cnot! |> toff!

invchuang (generic function with 1 method)

In [35]:
precircuit = [
    ('t', 1, 2, 3);
    ('c', 1, 2);
    ('n',1);
]

circuit = [
    ('t', 1, 2, 4);
    ('t', 3, 4, 5);
    ('n', 1);('n', 2);('n', 3);
    ('t', 1, 2, 6);
    ('t', 3, 6, 7);
    ('n',5);
    ('c',5,1);('c',5,2);('c',5,3);
    ('c',7,1);('c',7,2);('c',7,3)
    ]

circuit2 =
[
    ('n',4)
    ('t', 1, 2, 4);
    ('t', 1, 3, 4);
    ('t', 2, 3, 4);
    ('c',1,4);('c',2,4);('c',3,4);
    ('c',4,1);('c',4,2);('c',4,3);
    ]

a = [digits(Bool, i, base=2, pad=7) for i in 0:7]
for i in a
    print(docircuit!(i,circuit),"\n")
end

a = [digits(Bool, i, base=2, pad=4) for i in 0:7]
for i in a
    print(docircuit!(i,circuit2),"\n")
end

Bool[1, 1, 1, 0, 1, 1, 1]
Bool[1, 0, 0, 0, 1, 0, 0]
Bool[0, 1, 0, 0, 1, 0, 0]
Bool[1, 1, 0, 1, 1, 0, 0]
Bool[0, 0, 1, 0, 1, 1, 0]
Bool[1, 0, 1, 0, 1, 0, 0]
Bool[0, 1, 1, 0, 1, 0, 0]
Bool[0, 0, 0, 1, 0, 0, 0]
Bool[1, 1, 1, 1]
Bool[1, 0, 0, 0]
Bool[0, 1, 0, 0]
Bool[1, 1, 0, 0]
Bool[0, 0, 1, 0]
Bool[1, 0, 1, 0]
Bool[0, 1, 1, 0]
Bool[0, 0, 0, 1]


In [153]:
a = [digits(Bool, i, base=2, pad=4) for i in 0:7]
for i in a
    print(docircuit!(i,[precircuit;circuit2]),"\n")
end 

Bool[1, 0, 0, 0]
Bool[0, 1, 0, 0]
Bool[1, 1, 0, 0]
Bool[0, 0, 1, 0]
Bool[1, 0, 1, 0]
Bool[0, 1, 1, 0]
Bool[0, 0, 0, 1]
Bool[1, 1, 1, 1]


In [37]:
a = [digits(Bool, i, base=2, pad=4) for i in 0:7]
for i in a
    print(docircuit!(i,
            "n4t1.2.4t1.3.4t2.3.4c1.4c2.4c3.4c4.1c4.2c4.3"),"\n")
end 

Bool[1, 1, 1, 1]
Bool[1, 0, 0, 0]
Bool[0, 1, 0, 0]
Bool[1, 1, 0, 0]
Bool[0, 0, 1, 0]
Bool[1, 0, 1, 0]
Bool[0, 1, 1, 0]
Bool[0, 0, 0, 1]
